# Ensemble run

Here, we will investigate what the ensemble represents in a Monte-Carlo approach. See lecture "Introduction to Data Assimilation" for more information on the ensemble approach.

## Setting up the ensemble run

As a basis for the ensemble run, we will use a similar setting to the single run in the previous exercise (i.e. definition of model parameter, timestep, net precipitation data and initial storage value). **Please copy-paste the missing lines from Exercise 1.**

In [ ]:
import numpy as np

# Define parameter K  (copied from Exercise 1)
############ TO BE FILLED ##################

# Define timestep (copied from Exercise 1)
############ TO BE FILLED ##################

# Precipitation data (copied from Exercise 1)
############ TO BE FILLED ##################

# Initialize storage vector (copied from Exercise 1)
############ TO BE FILLED ##################


To generate a model ensemble, first, we have to define the ensemble size.

In [ ]:
# Ensemble size
Ne = 30  # ensemble size

We also need to load some Gaussian noise files, which we will use to perturb the input data, parameters and initial water storage state. **[Marker for extra exercises].**

In [ ]:
# Load Gaussian noise files
noise_K = np.load('data/noise_K.npy')       # shape: (1, Ne)
noise_S0_ens = np.load('data/noise_S0_ens.npy')
noise_mP = np.load('data/noise_mP.npy')

Here, the properties for the Gaussian noise are defined, and then used to perturb the different variables:

In [ ]:
# Set noise amplitude
Kmin, Kmax = 0.01, 0.99   # min and max value of model parameter
S0min, S0max = 2, 8       # min and max value of initial state
Pmin, Pmax = 0.8, 1.2     # multiplicator

# Perturb variables
K_ens = Kmin + (Kmax - Kmin) * noise_K      # shape: (Ne,)
S0_ens = S0min + (S0max - S0min) * noise_S0_ens  # shape: (Ne,)
mP = Pmin + (Pmax - Pmin) * noise_mP        # shape: (Ne,)
# Create precipitation ensemble (len(P) x Ne)
P_ens = np.tile(P.reshape(-1, 1), (1, Ne)) * np.tile(mP.reshape(1, -1), (len(P), 1))

## Initialization of ensemble spread

Before starting with the ensemble run, we need to launch a "warm-up" run to make sure that the model storage state has an appropriate ensemble spread that reflects the uncertainties of (1) initial storage; (2) precipitation and (3) model parameters. Let us run a warm-up over a period of 12 timesteps:

In [ ]:
from functions import linearModel
ini = [0, 12]  # Python index: 0-based, first 12 steps

S_ini = np.zeros((ini[1], Ne))
S_ini[0, :] = S0_ens

for ii in range(Ne):
    _, S_ini[ini[0]:ini[1], ii] = linearModel.linearModel(S_ini[ini[0]:ini[1], ii], 
                                              P_ens[ini[0]:ini[1], ii], 
                                              K_ens[0,ii], Dt)


## Ensemble run

We will now initialize the final ensemble run with the ouput of the "warm-up" run. The final ensemble run will be performed over a period of 24 timesteps.

In [ ]:
sim = [0, 24]  # first 24 steps

S_ens = np.zeros((sim[1], Ne))
S_ens[0, :] = S_ini[-1, :]

Q_ens = np.zeros((sim[1], Ne))

We are now ready to launch the ensemble run. We will iterate and run the linearModel function for each ensemble member.

In [ ]:
for ii in range(Ne):
    Q_ens[sim[0]:sim[1], ii], S_ens[sim[0]:sim[1], ii] = linearModel.linearModel(
        S_ens[sim[0]:sim[1], ii],
        P_ens[sim[0]:sim[1], ii],
        K_ens[0,ii],
        Dt
    )

## Visualization

Let us visualize the ensemble run results.

In [ ]:
from functions.Visualization_functions import F_Visualize_Ex2  # assuming we will translate this function later
F_Visualize_Ex2.F_Visualize_Ex2(P_ens, S_ens, Q_ens, sim)

## Extra exercises

- Go back to the "Marker for extra exercises". Triple the precipitation uncertainty (variable noise_mP) and run the whole script to see what happens.
- At the same place, reduce the ensemble size from 30 to 5. You can do this by modifying the size of the loaded noise variables (noise_K, noise_mP and  noise_S0_ens). Run the whole script to see what happens.

## Reflection questions

1. What does the ensemble spread represent? 


2. How does the uncertainty of S and R depend on the uncertainty of P – ET? 


3. What could be the benefits and challenges of increasing the ensemble? 

